In [1]:
import pyodbc
import pandas as pd

In [2]:
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = r'DESKTOP-L3GBMQ5\SQLEXPRESS'
DATABASE_NAME = 'Healthcare_Case_Study'

connection_string = (
    f"DRIVER={{{DRIVER_NAME}}};"
    f"SERVER={SERVER_NAME};"
    f"DATABASE={DATABASE_NAME};"
    f"Trusted_Connection=yes;"
)

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
print("Connected successfully!")

Connected successfully!


KPI Summary table

In [14]:
cursor.execute(
    """
    IF OBJECT_ID('dbo.Patients', 'U') IS NOT NULL
    DROP TABLE Hospital_KPI_Summary;
    
    DROP TABLE IF EXISTS Hospital_KPI_Summary;
    
    SELECT
        Department
        ,COUNT(Patient_ID) AS Total_Patients
        ,ROUND(AVG(Length_of_Stay), 2) AS Avg_Length_of_Stay
        ,ROUND(AVG(Wait_Time_Minutes), 2) AS Avg_Wait_Time
        ,ROUND(AVG(Satisfaction_Score), 2) AS Avg_Satisfaction_Score
        ,ROUND(SUM(Total_Revenue), 2) AS Total_Revenue
        ,ROUND(SUM(Treatment_Cost), 2) AS Total_Treatment_Cost
        ,ROUND(SUM(Profit), 2) AS Total_Profit
        ,ROUND(SUM(CAST(Readmission_30Days AS FLOAT))*100.0 / COUNT(*),2) AS Readmission_Rate_Percent
        ,ROUND(SUM(Total_Revenue)/COUNT(Patient_ID),2) AS Revenue_Per_Patient
        ,ROUND(SUM(Profit)/COUNT(Patient_ID),2) AS Profit_Per_Patient
    INTO
        Hospital_KPI_summary
    FROM
        Hospital_Operations_Cleaned_2
    GROUP BY
        Department;
    """
)

In [6]:
conn.commit()

Display Result

In [16]:
df_kpi = pd.read_sql("SELECT * FROM Hospital_KPI_Summary ORDER BY Total_Patients DESC;", conn)
df_kpi.head(10)

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_12652\2927823847.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_kpi = pd.read_sql("SELECT * FROM Hospital_KPI_Summary ORDER BY Total_Patients DESC;", conn)


,Department,Total_Patients,Avg_Length_of_Stay,Avg_Wait_Time,Avg_Satisfaction_Score,Total_Revenue,Total_Treatment_Cost,Total_Profit,Readmission_Rate_Percent,Revenue_Per_Patient,Profit_Per_Patient
0,Oncology,266,6.64,92.04,5.54,663944.74,514081.54,149863.20,0.0,2496.03,563.40
1,Ortho,136,7.01,86.28,5.54,325237.64,260318.38,64919.27,0.0,2391.45,477.35
2,Cardio,134,7.03,93.78,5.13,320094.76,249667.53,70427.23,0.0,2388.77,525.58
3,Pediatrics,129,7.48,93.64,5.36,324494.16,260318.64,64175.52,0.0,2515.46,497.48
4,Pediatric,127,7.11,88.24,5.09,295297.10,227956.76,67340.34,0.0,2325.17,530.24
5,Cardiology,124,7.85,83.87,5.53,309855.15,247569.60,62285.55,0.0,2498.83,502.30
6,Cardiology Dept.,120,7.38,92.15,5.11,295016.17,232060.06,62956.11,0.0,2458.47,524.63
7,Emergency,113,6.87,92.02,5.66,285050.73,229394.27,55656.46,0.0,2522.57,492.54
8,Orthopedics,103,8.03,94.15,5.40,234353.01,180526.38,53826.63,0.0,2275.27,522.59
